In [1]:
import sys
sys.path.append('..')
import dataset
import logging
import mxnet as mx
from mxnet import autograd
from mxnet import gluon, image, nd, init, metric
import model
import utils
from dataset import load_data_ChestX_ray14
from mxnet.gluon import nn
import matplotlib.pyplot as plt

C:\Users\weij\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
lr = 0.001
num_epoches = 100
momentum = 0.5
ctx = utils.try_all_gpus()

In [4]:
train_iter, test_iter = load_data_ChestX_ray14(16, resize=224)

utils.set_logging_level(level=logging.WARNING)

In [5]:
tuning_net = gluon.nn.HybridSequential()
dense121_net = gluon.model_zoo.vision.densenet121(pretrained=True, ctx=ctx)
output_layer = nn.Dense(14, weight_initializer=init.Xavier())
output_layer.initialize(ctx=ctx)
with tuning_net.name_scope():
    tuning_net.add(dense121_net.features,
                   output_layer)

net = tuning_net


In [9]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': momentum})
# loss_fn = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=False)
loss_fn = gluon.loss.SigmoidBinaryCrossEntropyLoss()

In [19]:
utils.evaluate_AUC(test_iter, net, ctx)

array([0.55484685, 0.50194799, 0.5326838 , 0.52455076, 0.52703218,
       0.49324393, 0.50361379, 0.52093742, 0.5931017 , 0.49582795,
       0.49158288, 0.48221518, 0.53898346, 0.44441851])

In [10]:
utils.train_parallel(train_iter, test_iter, net, loss_fn, trainer, ctx, num_epochs=10 )
net.save_params

training on  [gpu(0)]
epoch 1, loss 0.1572, time 3621.3 sec
train AUC:
 [0.68801702 0.56692316 0.77517799 0.61928136 0.6215412  0.57893043
 0.60096263 0.67863215 0.70436831 0.72163507 0.63921103 0.62581466
 0.63223255 0.55707679]
test AUC:
 [0.63774572 0.53946641 0.7159863  0.61678987 0.58921726 0.59446969
 0.56861382 0.67610673 0.65037965 0.6830881  0.59823752 0.66163105
 0.63872679 0.51727777]
epoch 2, loss 0.1503, time 3818.3 sec
train AUC:
 [0.73865639 0.64340725 0.82955123 0.65280902 0.67756301 0.62689897
 0.65047835 0.7318118  0.75113819 0.81892877 0.72325006 0.69131839
 0.69019383 0.62377505]
test AUC:
 [0.67211853 0.59168646 0.7403863  0.63386934 0.63292643 0.62204172
 0.57356688 0.72618446 0.66916931 0.73625432 0.64958913 0.7020349
 0.65456416 0.57192802]
epoch 3, loss 0.1468, time 3830.8 sec
train AUC:
 [0.75883837 0.71555465 0.84808183 0.66888503 0.69999052 0.65395809
 0.67886626 0.75850799 0.76636687 0.84645341 0.74719861 0.71050744
 0.71602908 0.66478013]
test AUC:
 [0.691

KeyboardInterrupt: 

In [15]:
net.save_params(str(14)+'denset121_07.params')

In [ ]:
utils.train_parallel(train_iter, test_iter, net, loss_fn, trainer, ctx, num_epochs=10 )
net.save_params(str(24)+'denset121_07.params')

training on  [gpu(0)]
epoch 1, loss 0.1275, time 3787.7 sec
train AUC:
 [0.83851758 0.90801938 0.9051784  0.76440276 0.84390292 0.78713474
 0.78517867 0.87056704 0.83624014 0.91526765 0.88814279 0.82389948
 0.82619135 0.85424683]
test AUC:
 [0.72620108 0.8244222  0.79944755 0.66300784 0.73641503 0.68369942
 0.6434441  0.81908407 0.69971467 0.81272664 0.82063697 0.74785677
 0.71591487 0.78071595]


In [ ]:
utils.train_parallel(train_iter, test_iter, net, loss_fn, trainer, ctx, num_epochs=10 )
net.save_params(str(34)+'denset121_07.params')

In [ ]:
utils.train_parallel(train_iter, test_iter, net, loss_fn, trainer, ctx, num_epochs=10 )
net.save_params(str(44)+'denset121_07.params')